In [2]:
import pandas as pd; pd.options.mode.chained_assignment = None
import numpy as np

# Built In Imports
from datetime import datetime
from glob import glob
import warnings
import IPython
import urllib
import zipfile
import pickle
import shutil
import string
import math
import tqdm
import time
import os
import gc
import re

# Visualization Imports
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from PIL import Image
import matplotlib
import plotly
import PIL
import cv2
from pycocotools.coco import COCO

# Other Imports
from tqdm.notebook import tqdm
import bbox_visualizer as bbv

LABEL_COLORS = [px.colors.label_rgb(px.colors.convert_to_RGB_255(x)) for x in sns.color_palette("Spectral", 10)]
LABEL_COLORS_WOUT_NO_FINDING = LABEL_COLORS[:8]+LABEL_COLORS[9:]

In [3]:
root_dir = '/opt/ml/dataset/'
train_gt_path = '/opt/ml/dataset/train.json'

In [4]:
coco = COCO(train_gt_path)

train_df = pd.DataFrame()

image_ids = []
class_name = []
class_id = []
x_min = []
y_min = []
x_max = []
y_max = []
area = []
object_id = []
classes = ["General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]


for image_id in coco.getImgIds():

    image_info = coco.loadImgs(image_id)[0]
    ann_ids = coco.getAnnIds(imgIds=image_info['id'])
    anns = coco.loadAnns(ann_ids)
        
    file_name = image_info['file_name']
        
    for ann in anns:
        image_ids.append(file_name)
        class_name.append(classes[ann['category_id']])
        class_id.append(ann['category_id'])
        x_min.append(float(ann['bbox'][0]))
        y_min.append(float(ann['bbox'][1]))
        x_max.append(float(ann['bbox'][0]) + float(ann['bbox'][2]))
        y_max.append(float(ann['bbox'][1]) + float(ann['bbox'][3]))
        area.append(float(ann['bbox'][2]) * float(ann['bbox'][3]))
        object_id.append(int(ann['id']))

train_df['image_id'] = image_ids
train_df['class_name'] = class_name
train_df['class_id'] = class_id
train_df['x_min'] = x_min
train_df['y_min'] = y_min
train_df['x_max'] = x_max
train_df['y_max'] = y_max
train_df['area'] = area
train_df['obj_id'] = object_id

print(len(train_df))

loading annotations into memory...
Done (t=0.23s)
creating index...
index created!
23144


In [5]:
# mAP_s: < 32^2
# mAP_m: 32^2 < area < 96^2
# mAP_l: > 96^2
# Unit: number of pixels
limits = [32**2, 96**2]
print(len(train_df))
train_df.head()

23144


,image_id,class_name,class_id,x_min,y_min,x_max,y_max,area,obj_id
0,train/0000.jpg,General trash,0,197.6,193.7,745.4,663.4,257301.66,0
1,train/0001.jpg,Metal,3,0.0,407.4,57.6,588.0,10402.56,1
2,train/0001.jpg,Plastic bag,7,0.0,455.6,144.6,637.2,26259.36,2
3,train/0001.jpg,Glass,4,722.3,313.4,996.6,565.3,69096.17,3
4,train/0001.jpg,Plastic,5,353.2,671.0,586.9,774.4,24164.58,4


In [6]:
small, medium, large = 0, 0, 0
for i in tqdm(range(len(train_df))):
    if train_df['area'][i] <= limits[0]:
        small += 1
    elif train_df['area'][i] <= limits[1]:
        medium += 1
    else:
        large += 1

# 
total = len(train_df)
print(small / total, medium / total, large / total)

  0%|          | 0/23144 [00:00<?, ?it/s]

0.013524023505012098 0.2244210162461113 0.7620549602488766


In [12]:
fig = px.ecdf(train_df, x="area")
fig.show()